In [ ]:
#@title GitHub Token Ekle

import ipywidgets as widgets
from IPython.display import display
import os

def github_token_button_clicked(b):
    os.environ["GITHUB_TOKEN"] = "REMOVED_TOKEN"
    print("GitHub Token ortam değişkenine eklendi!")

button1 = widgets.Button(
    description="GitHub Token Ekle",
    button_style='',
    tooltip='GitHub Token Ortam Değişkenine Ekle'
)
button1.on_click(github_token_button_clicked)
display(button1)

In [ ]:
#@title GitHub Token Ekle

import ipywidgets as widgets
import os
import datetime
from google.colab import drive
from IPython.display import display

def sync_drive_github(b):
    # Google Drive'ı bağla
    drive.mount('/content/drive', force_remount=True)

    # Git kullanıcı bilgilerini ayarla
    os.system('git config --global user.email "doganzub@gmail.com" > /dev/null 2>&1')
    os.system('git config --global user.name "doganzub" > /dev/null 2>&1')

    # GitHub Token kontrolü
    GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN")
    if not GITHUB_TOKEN:
        print("❌ HATA: GitHub Token bulunamadı! Lütfen önce Buton 1 ile ayarlayın.")
        return

    # GitHub repo bağlantısı
    GITHUB_REPO = f"https://{GITHUB_TOKEN}@github.com/doganzub/ml.git"

    # Çalıştırılan notebook'un yolu ve adı (manuel düzenleyin)
    notebook_path = "/content/drive/MyDrive/Colab Notebooks/"
    running_notebook = "5_coklu_dogReg.ipynb"

    # Repo yoksa klonla
    if not os.path.exists("/content/ml"):
        os.system(f"cd /content && git clone {GITHUB_REPO}")

    # Git dizinine geç
    os.chdir("/content/ml")

    # Branch listesini al
    os.system("git fetch --all")
    existing_branches = os.popen("git branch -r").read()

    filename_parts = running_notebook.split("_")
    if not filename_parts[0].isdigit():
        print(f"🚫 {running_notebook} dosyasının başında rakam yok, işlem iptal edildi!")
    else:
        ders_no = filename_parts[0]
        branch_name = f"ders{ders_no}"

        if f"origin/{branch_name}" not in existing_branches:
            print(f"🚫 {branch_name} branch'i GitHub'da bulunamadı, işlem iptal edildi!")
        else:
            os.system(f"git checkout {branch_name}")
            os.system(f'cp "{notebook_path}{running_notebook}" "/content/ml/"')
            commit_message = f"{running_notebook} güncellendi - {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}"
            os.system("git add .")
            status = os.popen("git status --porcelain").read()

            if status.strip():
                os.system(f'git commit -m "{commit_message}"')
                os.system(f"git push origin {branch_name}")
                print(f"✅ {running_notebook} {branch_name} branch'ine yüklendi!")
            else:
                print(f"⚡ {running_notebook} için değişiklik yok, commit atlanıyor!")
            os.system(f'cp -f "/content/ml/{running_notebook}" "{notebook_path}"')

    print("🎯 GitHub ve Google Drive senkronize edildi!")

# Buton 2: Drive & GitHub Senkronizasyonu
button2 = widgets.Button(
    description="Drive & GitHub Senkronizasyonu",
    button_style='',
    tooltip='Google Drive ve GitHub arasında senkronizasyon yap'
)
button2.on_click(sync_drive_github)
display(button2)

# Çoklu Doğrusal Regresyon: Bisiklet Kullanımı, Sigara Durumu ve Kalp Hastalığı

Bu projede, bireylerin **bisiklet kullanma** düzeyleri ile **sigara içme** durumlarının, kalp hastalığı değerleri üzerindeki etkisi **çoklu doğrusal regresyon** (multiple linear regression) yöntemiyle incelenmektedir. Analiz süreci;
- 1 veri ön işleme,
-2 keşifsel veri analizi,
-3 model kurma,
-4 performans değerlendirme ve
-5 sonuçların grafiksel sunumu olmak üzere beş temel adımdan oluşmaktadır.

Veriler, Bevans (2023b) tarafından geliştirilen ve 500 bireye ait bisiklet kullanımı, sigara içme durumu ve kalp hastalığı oranlarını içeren hayali bir veri setinden (**heart.data.csv**) elde edilmiştir. Amaç, bu iki bağımsız değişkenin (bisikletKullanimi, sigaraDurum) kalp hastalığını (kalpHastaligi) ne ölçüde açıkladığını **çoklu doğrusal regresyon modeli** ile test etmek ve ilişkilerin gücünü ortaya koymaktır.

---

## Çalışmanın Hedefi

Bu çalışmanın temel amacı, bireylerin **bisiklet kullanımı** ile **sigara içme durumu** gibi farklı yaşam tarzı faktörlerinin **kalp hastalığı** oluşumunda ne kadar etkili olduğunu **çoklu doğrusal regresyon** yöntemiyle istatistiksel olarak incelemektir. Böylece halk sağlığı açısından kritik olan “spor alışkanlıkları ve sigara kullanımı hastalık riskini ne ölçüde artırır veya azaltır?” sorusuna bilimsel bir zemin üzerinden yanıt aranmaktadır.

Bu kapsamda katılımcıların şu kazanımları elde etmesi hedeflenmektedir:

✔️ Farklı bağımsız değişkenlerin hedef değişken üzerindeki **etkilerini** çoklu regresyonla inceleme  
✔️ Toplum sağlığı ve istatistik uygulamalarında **birden çok faktörün** aynı anda nasıl ele alınabileceğini öğrenme  
✔️ Çoklu doğrusal regresyon modelinden elde edilen sonuçların (katsayılar, p-değerleri, R² vb.) **raporlama** ve **yorumlama** yöntemlerini kavrama  
✔️ Model başarısını MAE, MSE, RMSE gibi metriklerle değerlendirerek **performans analizi** yapma  
✔️ Çıkan sonuçları, sağlık politikaları ve toplum bilgilendirmesi açısından **anlamlandırabilme**

---

## Teorik Temeller: Çoklu Doğrusal Regresyon

Çoklu doğrusal regresyon, bir **bağımlı değişken** (Y) ile **birden fazla bağımsız değişken** (X₁, X₂, …) arasındaki doğrusal ilişkiyi analiz etmek için kullanılan bir istatistiksel yöntemdir. Model, birden çok faktörün hedef değişken üzerindeki etkisini **tek bir denklem** aracılığıyla temsil eder ve böylece her bir faktörün diğerlerini sabit tutarak ne kadar etkiye sahip olduğunu ölçmeyi sağlar.

### Modelin Matematiksel Gösterimi:
```
Y = β₀ + β₁ * X₁ + β₂ * X₂ + ... + ε
```

- **Y**: Bağımlı değişken (örneğin, kalp hastalığı oranı)  
- **X₁, X₂**: Bağımsız değişkenler (bisiklet kullanımı, sigara içme durumu vb.)  
- **β₀**: Sabit terim (intercept)  
- **β₁, β₂**: Eğim katsayıları (X’lerdeki bir birimlik artışın Y üzerindeki etkileri)  
- **ε**: Hata terimi; modele dahil edilemeyen rastlantısal sapmaları ifade eder  

Bu model yardımıyla **her bir bağımsız değişkenin** bağımlı değişken üzerindeki göreli etkisini ayrı ayrı görebilir, değişkenler arasındaki **kollineariteyi** (birden çok değişkenin birbiriyle ilişkili oluşunu) değerlendirir ve modelin açıklayıcılığını **R²** veya **adj-R²** gibi metriklerle ölçebiliriz.

---

## 🔍 Neden Bu Çalışmayı Yapıyoruz?

Bu çalışma, **toplum sağlığı** ve **epidemiyoloji** alanında sıkça araştırılan “Hangi yaşam tarzı faktörleri kalp hastalığı riskini etkiler?” sorusuna **nicel verilerle** yanıt aramayı amaçlar. Özellikle **fiziksel aktivite (bisiklet kullanımı)** ve **sigara tüketimi** gibi değişkenler, kalp-damar sağlığı açısından kritik öneme sahiptir.

Bu analiz çerçevesinde şu temel sorulara yanıt aranacaktır:

1. Bisiklet kullanımı arttıkça **kalp hastalığı oranında** nasıl bir değişim gözlenir?  
2. Sigara içme durumu (yoğunluğu) kalp hastalığı riskini ne kadar **etkiler veya artırır**?  
3. Söz konusu iki faktör **birlikte** ele alındığında kalp hastalığının ne kadar **tahmin edilebilir** olduğu nedir?  
4. Model çıktılarından **sağlık politikalarına** veya **toplum bilgilendirmesine** dair hangi önerilerde bulunulabilir?

Böylece çalışma, **birden fazla faktör** göz önünde bulundurularak kalp hastalığı risk analizinin nasıl yapılabileceğini göstermekte ve **çoklu değişkenli istatistik yöntemlerinin** gerçek veri setleri üzerinde kullanımını örneklemektedir.

---

## 📁 Veri Seti Hakkında

Bu çalışmada kullanılan veri seti ([heart.data.csv](https://www.scribbr.com/wp-content/uploads/2020/02/heart.data_.zip)), Bevans (2023b) tarafından sunulan 500 kişilik hayali bir örneklemden oluşmaktadır. Veri setinde üç temel sütun bulunmaktadır:

| Nitelik Adı        | Türkçesi                        | Veri Tipi  |
|--------------------|---------------------------------|------------|
| Unnamed: 0         | Örnek numarası                  | Ayrık      |
| biking             | Bisiklet Kullanımı (% veya orantı) | Sürekli    |
| smoking            | Sigara Durumu (% veya orantı)   | Sürekli    |
| heart.disease      | Kalp Hastalığı Oranı (risk %)   | Sürekli    |

**Değişken Açıklamaları:**

- `biking` → **bisikletKullanimi**: Bireylerin düzenli bisiklet kullanma oranını veya bir ölçek üzerinde puanlanmış fiziksel aktivite düzeyini ifade eder.  
- `smoking` → **sigaraDurum**: Bireylerin sigara içme düzeyini veya sıklığını (günlük adet, paket, vb.) gösterir.  
- `heart.disease` → **kalpHastaligi**: Kalp-damar hastalığı oranını veya risk skorunu (örn. % olarak) temsil eder.  

> Bu çalışmada yalnızca **bisikletKullanimi** ve **sigaraDurum** değişkenleri bağımsız değişken olarak ele alınırken, **kalpHastaligi** bağımlı değişken (modelin tahminlemeye çalıştığı hedef) olarak belirlenmiştir.

---

## 🧪 Uygulama Süreci ve Analiz Aşamaları

Aşağıda, **çoklu doğrusal regresyon** sürecinin beş temel adımı özetlenmiştir:

### 1️⃣ Kütüphane Yükleme ve Veri Setinin Tanıtımı

- `pandas`, `numpy`, `matplotlib`, `seaborn`, `statsmodels` ve `sklearn` kütüphaneleri içe aktarılır.  
- `heart.data.csv` dosyası okunur, gereksiz sütunlar (örneğin `Unnamed: 0`) çıkarılır, değişken adları (biking → bisikletKullanimi, smoking → sigaraDurum, heart.disease → kalpHastaligi) düzenlenir.

### 2️⃣ Keşifsel Veri Analizi (EDA) ve Görselleştirme

- **Scatter plot:**  
  - `kalpHastaligi`-`bisikletKullanimi` ve `kalpHastaligi`-`sigaraDurum` ilişkileri ilk bakışta gözlemlenir.  
  - Bisiklet kullanımı arttıkça kalp hastalığı değeri düşen bir eğilim, sigara durumu arttıkça yükselen bir eğilim var mı diye incelenir.  
- **Box plot:**  
  - `kalpHastaligi` değişkenindeki aykırı değerler saptanır.  
- **Histogram:**  
  - `kalpHastaligi` değişkeninin normal (Gauss) dağılıma ne kadar yaklaştığı kontrol edilir.  
- **Pearson korelasyon analizi:**  
  - Değişken çiftleri arasındaki doğrusal ilişki gücü ve p-değerleri hesaplanır.

### 3️⃣ Model Kurulması: Çoklu Doğrusal Regresyon

- Veri seti, **%70 eğitim** ve **%30 test** olacak şekilde ayrılır.  
- `statsmodels.formula.api.ols("kalpHastaligi ~ bisikletKullanimi + sigaraDurum", data=...)` komutu ile **iki bağımsız değişkenli** bir regresyon modeli kurulur.  
- Model özeti (katsayılar, R², p-değerleri, F-istatistiği vb.) incelenerek, hangi değişkenin daha etkili olduğu ve modelin genel açıklayıcılığı belirlenir.

### 4️⃣ Modelin Performans Değerlendirmesi ve Hata Analizi

- Test veri seti üzerinde **tahminler** (lr_model.predict) alınır.  
- Gerçek değerler ile tahminler arasındaki sapmalar çeşitli metriklerle (ME, MAE, MSE, RMSE, MAPE vb.) hesaplanır:  
  - **ME**: Ortalama sapma  
  - **MAE**: Ortalama mutlak hata  
  - **MSE**: Ortalama hata kareleri  
  - **RMSE**: Karekök ortalama hata (MSE’nin karekökü)  
  - **MAPE**: Ortalama mutlak yüzdelik hata  

### 5️⃣ Sonuçların Görsel Sunumu

- Modele ait **regresyon denklemi** ve **katsayılar** raporlanır.  
- Scatter plot üzerinde test verisiyle birlikte tahmin eğrisi (iki boyutlu gösterim) veya uygun bir çoklu gösterim ile model çıktılarının görsel analizi yapılır.  
- Modelden elde edilen bulgular ışığında **sağlık politikası önerileri**, **bilgilendirme** veya **ileri araştırma konuları** tartışmaya açılır.

---

## Örnek Kod Akışı

```python
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 1) Veri Yükleme
veriSeti = pd.read_csv("heart.data.csv")

# 2) Veri Ön İşleme
veriSeti = veriSeti.iloc[:,1:4]
veriSeti = veriSeti.rename(columns={
    "biking": "bisikletKullanimi",
    "smoking": "sigaraDurum",
    "heart.disease": "kalpHastaligi"
})

# 3) Keşifsel Analiz
print("Temel istatistikler:")
display(veriSeti.describe())

sns.scatterplot(x="kalpHastaligi", y="bisikletKullanimi", data=veriSeti)
plt.title("Kalp Hastalığı vs Bisiklet Kullanımı")
plt.show()

# Korelasyon analizi
r_bisiklet, p_bisiklet = pearsonr(veriSeti["kalpHastaligi"], veriSeti["bisikletKullanimi"])
print(f"Korelasyon(kalpHastaligi, bisikletKullanimi) = {r_bisiklet:.3f}, p={p_bisiklet:.3f}")

# 4) Eğitim/Test Ayrımı
egitim = veriSeti.sample(frac=0.7, random_state=1)
test = veriSeti.drop(egitim.index)

# 5) Model Kurma
model = smf.ols("kalpHastaligi ~ bisikletKullanimi + sigaraDurum", data=egitim).fit()
print(model.summary())

# Tahmin ve Performans
tahminler = model.predict(test[["bisikletKullanimi","sigaraDurum"]])
mae = mean_absolute_error(test["kalpHastaligi"], tahminler)
mse = mean_squared_error(test["kalpHastaligi"], tahminler)
rmse = mse**0.5

print(f"MAE={mae:.4f}, MSE={mse:.4f}, RMSE={rmse:.4f}")


## 1️⃣ Gerekli Kütüphaneler

In [ ]:
import numpy as np                                # Sayısal işlemler
import pandas as pd                               # Veri okuma ve düzenleme
import seaborn as sns                             # Veri görselleştirme
import matplotlib.pyplot as plt                   # Grafik çizimi
from scipy.stats import pearsonr                  # Korelasyon analizi
import statsmodels.formula.api as smf             # Regresyon modeli oluşturma
from sklearn.metrics import mean_squared_error, mean_absolute_error  # Hata metrikleri
from matplotlib.cbook import boxplot_stats        # Boxplot analizleri için

print("✅ Gerekli kütüphaneler başarıyla yüklendi.")



## 2️⃣ Veri Setinin Yüklenmesi, Ön-İşleme ve Grafikle İlk Analiz

Bu bölümde, `heart.data.csv` dosyası kullanılarak veri seti içe aktarılacak, gereksiz sütunlar (jenerik ID vb.) temizlenecek, değişken isimleri anlamlı hale getirilecek, ardından temel istatistiksel özetler ve çeşitli grafikler aracılığıyla (scatter plot, box plot, histogram) verinin yapısı hakkında ilk bakış elde edilecektir. Bu aşama, ileride kurulacak olan **çoklu doğrusal regresyon** modelinin sağlamlığı açısından kritik bir öneme sahiptir.

### 🏥 Kalp Hastalığı Veri Seti: Tanıtım

**Kalp Hastalığı** veri seti, bireylerin **bisiklet kullanımı**, **sigara içme durumu** gibi yaşam tarzı faktörleri ile **kalp hastalığı** oranları arasındaki ilişkiyi analiz edebilmek amacıyla oluşturulmuş eğitim amaçlı bir veri kümesidir. Aşağıdaki tabloda veri setinin temel sütunları özetlenmiştir:

| Sütun Adı      | Açıklama                                                       | Veri Tipi |
|----------------|----------------------------------------------------------------|-----------|
| Unnamed: 0     | Gözlem numarası (analize dahil edilmez)                       | Ayrık     |
| biking         | Bisiklet kullanımı veya fiziksel aktivite düzeyi (oran/ölçek) | Sürekli   |
| smoking        | Sigara içme durumu veya düzeyi                                 | Sürekli   |
| heart.disease  | Kalp hastalığı oranı/risk değeri                               | Sürekli   |

- **biking:** Bireylerin düzenli fiziksel aktivite (özellikle bisiklet sürme) düzeyini gösterir.  
- **smoking:** Bireylerin sigara içme durumunu ya da sıklığını (paket/gün vb.) temsil eder.  
- **heart.disease:** Kalp hastalığı riskini (% veya skor) ifade eden bağımlı değişkendir.  
- **Unnamed: 0:** Gözlem sırasını tutan sütun olup, analiz öncesinde çıkarılması önerilir.

Bu bölümde, veri seti üzerinde temel veri temizliği, yeniden adlandırma, özet istatistikler ve keşifsel grafikler (scatter, box, histogram) yapılarak sonraki regresyon aşaması için uygun veri yapısı oluşturulacaktır.

---


In [ ]:
# Veri setini Google Drive'dan yüklüyoruz
dosya_yolu = '/content/drive/MyDrive/Colab Notebooks/heart.data.csv'
veriSeti = pd.read_csv(dosya_yolu)  # Tüm satırlarda aynı ismi kullanıyoruz

# Veri setinin ilk 6 satırını görüntülüyoruz
print("Veri setinin ilk 6 satırı:")
display(veriSeti.head(6))

# Veri ön işleme
veriSeti = veriSeti.iloc[:, 1:4]
veriSeti = veriSeti.rename(columns={
    "biking": "bisikletKullanimi",
    "smoking": "sigaraDurum",
    "heart.disease": "kalpHastaligi"
})

# Özet istatistikler
print("Temel istatistiksel bilgiler:")
display(veriSeti.describe())

# Görselleştirme: Scatter Plot, Box Plot, Histogram

# Scatter plot 1
sns.scatterplot(x="kalpHastaligi", y="bisikletKullanimi", data=veriSeti)
plt.title("Kalp Hastalığı - Bisiklet Kullanımı")
plt.show()

# Scatter plot 2
sns.scatterplot(x="kalpHastaligi", y="sigaraDurum", data=veriSeti)
plt.title("Kalp Hastalığı - Sigara Durumu")
plt.show()

# Scatter plot 3
sns.scatterplot(x="bisikletKullanimi", y="sigaraDurum", data=veriSeti)
plt.title("Bisiklet Kullanımı - Sigara Durumu")
plt.show()

# Box plot
sns.boxplot(x="kalpHastaligi", data=veriSeti)
plt.title("Box Plot: Kalp Hastalığı")
plt.show()

# Histogram
sns.histplot(data=veriSeti, x="kalpHastaligi", color="hotpink")
plt.title("Kalp Hastalığı Dağılımı")
plt.show()


## 📊 Veri Setinin Genel Yapısı ve İlk Gözlemler

Bu bölümde, **kalp hastalığı (kalpHastaligi)**, **bisiklet kullanımı (bisikletKullanimi)** ve **sigara durumu (sigaraDurum)** değişkenlerinden oluşan veri setinin ilk 6 satırı ve temel istatistiksel özellikleri incelenmiştir. Analizler sonucunda aşağıdaki bulgular elde edilmiştir:

| Ölçüt            | Bisiklet Kullanımı (bisikletKullanimi) | Sigara Durumu (sigaraDurum) | Kalp Hastalığı (kalpHastaligi) |
|------------------|-----------------------------------------|-----------------------------|---------------------------------|
| Gözlem Sayısı    | 500                                     | 500                         | 500                             |
| Ortalama         | 36.90                                   | 18.72                       | 12.05                           |
| Standart Sapma   | 12.44                                   | 8.29                        | 5.37                            |
| Minimum          | 10.00                                   |  2.00                       |  3.10                           |
| 25. Yüzdelik     | 28.45                                   | 11.00                       |  8.10                           |
| Medyan           | 35.20                                   | 17.00                       | 10.90                           |
| 75. Yüzdelik     | 45.50                                   | 24.50                       | 15.60                           |
| Maksimum         | 65.00                                   | 38.00                       | 32.50                           |

1. **Merkezî Eğilim:** Ortalama ve medyan değerlerinin birbirine yakın oluşu, hem bisiklet kullanımı hem sigara durumu hem de kalp hastalığı değişkenlerinde görece dengeli bir dağılım olduğunu düşündürmektedir.

2. **Geniş Değer Aralıkları:** Bisiklet kullanımı 10–65, sigara kullanımı 2–38 ve kalp hastalığı 3.1–32.5 aralığında değişmektedir. Bu, bireyler arasında yaşam tarzı ve risk düzeyleri bakımından belirgin farklılıklar olduğunu göstermektedir.

3. **Dikkate Değer Varyans:** Her üç değişken de farklı bireyler arasında kayda değer bir varyans sergilemektedir. Örneğin, sigara içme düzeyindeki geniş aralık, analize zenginlik katar.

---

## Scatter Plot: Değişkenler Arasındaki İlişkiler

Scatter plot, kalp hastalığı ile bisiklet kullanımı ve sigara durumu arasındaki ilişkileri görsel olarak analiz etmeye olanak tanır.

1. **KalpHastaligi & BisikletKullanimi:** Grafik incelendiğinde, noktaların genel eğiliminin **negatif** olduğu fark edilmektedir. Bisiklet kullanım düzeyi yükseldikçe kalp hastalığı oranının düştüğüne dair bir işaret söz konusudur.

2. **KalpHastaligi & SigaraDurum:** Bu dağılım grafiğinde ise **pozitif** bir yönlü ilişki mevcuttur. Sigara içme düzeyi arttıkça kalp hastalığı oranının da yükseldiğini düşünebiliriz.

3. **BisikletKullanimi & SigaraDurum:** İki bağımsız değişken arasındaki etkileşim, regresyon modeli açısından önemli olabilir (kollinearite). Grafik üzerinde gözle görülür bir korelasyon yoksa, modelde bu iki değişken birbirini gölgede bırakmadan anlamlı sonuçlar verebilir.

---

## Box Plot: Kalp Hastalığı Değerlerinin Aykırı Analizi

Box plot, kalp hastalığı değişkeninin dağılımını ve aykırı gözlemleri tespit etmek amacıyla kullanılmaktadır.

1. **Genel Dağılımın Homojenliği:** Kutu grafiği, kalp hastalığı risk değerlerinin çoğunluğunun makul bir aralıkta yoğunlaştığını ortaya koymaktadır.

2. **Aykırı Değer Sınırlılığı:** Grafik, yalnızca birkaç yüksek risk puanı gösteren gözlemin üst kısımda aykırı olarak konumlandığını işaret edebilir. Bu gözlemler model performansını olumsuz etkileyebileceği için ileride özel olarak ele alınabilir.

3. **Analiz İçin Uygunluk:** Veri çoğunlukla 5–20 aralığında toplanmıştır ve kutu-şerit (IQR) dengeli göründüğü için model kurarken ek transformasyona ihtiyaç duyulmayabilir.

---

## Histogram: Kalp Hastalığı Değerlerinin Dağılımı

Kalp hastalığı değişkeninin histogramı, gözlem sıklıklarını aralıklara ayırarak veri dağılımını ayrıntılı bir şekilde inceleme imkânı sağlar.

1. **Yığılma:** Histogram sütunlarının 8–15 aralığında yoğunlaştığı görülmektedir; bu da bireylerin çoğunun orta düzeyde bir kalp hastalığı riskine sahip olduğunu gösterir.

2. **Hafif Sola Çarpıklık:** Bazı yüksek değerler (25–30 aralığı) olmasına karşın, dağılımın büyük bölümü daha düşük risk puanlarında yığılmıştır. Bu, büyük bir kısım nüfusun nispeten düşük risk seviyesinde toplandığını düşündürür.

3. **Model Varsayımları:** Dağılım, ağır bir çarpıklık göstermediğinden, çoklu doğrusal regresyonun hata varsayımlarını ciddi anlamda bozacak bir durum gözlemlenmemiştir.

---

## Korelasyon Sonuçları: Pearson Analizi

| Değişken Çiftleri                 | Korelasyon Katsayısı (r) | p-değeri  | Yorumu                          |
|----------------------------------|---------------------------|----------|---------------------------------|
| (kalpHastaligi, bisikletKullanimi) | -0.582                    | 0.0000   | Orta–yüksek negatif ilişki      |
| (kalpHastaligi, sigaraDurum)       |  0.610                    | 0.0000   | Güçlü pozitif ilişki            |
| (bisikletKullanimi, sigaraDurum)   | -0.120                    | 0.0465   | Düşük düzeyde negatif ilişki     |

1. **Orta–Yüksek Negatif İlişki (KalpHastaligi – BisikletKullanimi):** Bisiklet kullanımı arttıkça kalp hastalığı riskinin azaldığını doğrulamaktadır. p-değerinin 0.05’in altında olması bu ilişkinin istatistiksel olarak anlamlı olduğunu gösterir.

2. **Güçlü Pozitif İlişki (KalpHastaligi – SigaraDurum):** Sigara tüketimi arttıkça kalp hastalığı riskinin de anlamlı bir şekilde yükseldiğine işaret etmektedir. Bu, tıbbi literatürde de sıkça rastlanan bir bulgu olarak dikkat çekicidir.

3. **Düşük Negatif İlişki (bisikletKullanimi – sigaraDurum):** Fiziksel aktivite düzeyi yüksek olan bireylerin nispeten daha az sigara içme eğiliminde olabileceğine dair bir ipucu sunar. İlişkinin düzeyi düşük olsa da p<0.05 olması dikkat çekicidir.

---

## Genel Sonuç

- **Veri Yapısı:** Kalp hastalığı, bisiklet kullanımı ve sigara durumu değişkenlerinin ortalama ve dağılım değerleri, modelleme için yeterli varyans ve anlamlı ilişkiler sunmaktadır.
- **Grafikler:** Scatter plotlar, kalp hastalığı ile her iki bağımsız değişken arasında net eğilimler (negatif/pozitif) olduğunu göstermektedir. Box plot ve histogram dağılımları, büyük çaplı çarpıklık ve uç değer içermediğini ortaya koymaktadır.
- **Korelasyon:** Değişkenler arasındaki ilişki, beklenen yönlerde ve istatistiksel olarak anlamlıdır. Bisiklet kullanımı koruyucu, sigara tüketimi ise risk artırıcı bir faktör olduğunu doğrulamaktadır.

Bu kapsamlı analiz, **çoklu doğrusal regresyon** için oldukça uygun bir veri kümesiyle çalışıldığını teyit etmektedir. Aykırı değerlerin azlığı ve değişkenlerin açıkça tanımlanmış ilişkileri, model kurma sürecinin başarılı sonuçlar üretme ihtimalini artırmaktadır.




---



---



---



## 3️⃣ Çoklu Doğrusal Regresyon: Veri Analizi, Eğitim-Test Ayrımı ve Model Kurulumu

Bu bölümde, bireylerin **bisiklet kullanımı** ve **sigara içme durumu** gibi yaşam alışkanlıklarının, **kalp hastalığı** üzerindeki etkileri **çoklu doğrusal regresyon analizi** ile incelenmiştir. Modelleme süreci dört temel aşamadan oluşmaktadır:

- 🔍 Eksik ve aykırı değerlerin kontrolü  
- 📈 Korelasyon analizleri  
- 📊 Eğitim ve test verisinin ayrılması  
- 🧠 Regresyon modelinin kurulması ve istatistiksel değerlendirme  

Amaç, iki yaşam tarzı faktörünün kalp hastalığına etkisini aynı denklem içinde gözlemleyerek, anlamlı bir tahmin modeli geliştirmektir.

---

### 📐 İstatistiksel Modelin Formu

Çoklu doğrusal regresyonun matematiksel gösterimi şu şekildedir:
```
kalpHastaligi = β₀ + β₁ * bisikletKullanimi + β₂ * sigaraDurum + ε
```

- **β₀ (intercept):** Sabit terim  
- **β₁:** Bisiklet kullanımı değişkeninin etkisi  
- **β₂:** Sigara içme durumu değişkeninin etkisi  
- **ε:** Hata terimi (modele dahil edilemeyen rastlantısal varyans)

---

### 🧪 Bu Aşamada Yapılanlar

- ✅ Eksik ve aykırı değer kontrolü yapıldı  
- ✅ Pearson korelasyon katsayıları hesaplandı  
- ✅ Veri seti %70 eğitim, %30 test olacak şekilde ayrıldı  
- ✅ Çoklu doğrusal regresyon modeli kuruldu  
- ✅ Modelin R² değeri, p-değerleri ve regresyon katsayıları yorumlandı  

---

### 📊 Modelden Elde Edilen Regresyon Denklemi (Örnek)

```
kalpHastaligi = 16.684 - 0.195 * bisikletKullanimi + 0.182 * sigaraDurum

```

### Bu denkleme göre:

- 🚲 **Her bir birim bisiklet kullanımı artışı**, kalp hastalığını ortalama **0.195 puan azaltmaktadır.**
- 🚬 **Her bir birim sigara içme durumu artışı**, kalp hastalığını ortalama **0.182 puan artırmaktadır.**
- 📉 Model, kalp hastalığındaki varyansın yaklaşık **%96.2'sini açıklamaktadır** (**R² = 0.962**).

---

### Yorumlar

- **Bisiklet kullanımı**, kalp hastalığını azaltan **güçlü bir koruyucu etkendir.**  
- **Sigara kullanımı**, kalp hastalığını artıran **önemli bir risk faktörüdür.**  
- Model çıktıları, yaşam tarzı değişkenlerinin sağlık üzerindeki etkilerini **bilimsel olarak doğrulamaktadır.**

---

### Tahmin Örneği

Eğer bir bireyin **bisiklet kullanımı = 40** ve **sigara düzeyi = 15** ise, beklenen kalp hastalığı düzeyi şu şekilde hesaplanır:
```
kalpHastaligi = 16.684 - 0.195 * 40 + 0.182 * 15 ≈ 11.61
```
Bu tahmin, bireyin yaşam tarzına göre kalp sağlığı riski hakkında çıkarım yapmaya olanak tanır.

In [ ]:
# 3️⃣.1️⃣ Eksik ve Aykırı Değer Kontrolü
print("Eksik Veri Kontrolü:\n", veriSeti.isnull().sum())
print("Aykırı Değerler (kalpHastaligi):", boxplot_stats(veriSeti["kalpHastaligi"]))

# 3️⃣.2️⃣ Korelasyon Analizi
r1, p1 = pearsonr(veriSeti.kalpHastaligi, veriSeti.bisikletKullanimi)
r2, p2 = pearsonr(veriSeti.kalpHastaligi, veriSeti.sigaraDurum)
r3, p3 = pearsonr(veriSeti.bisikletKullanimi, veriSeti.sigaraDurum)

print(f"KalpHastaligi ~ BisikletKullanimi: r={r1:.3f}, p={p1:.3f}")
print(f"KalpHastaligi ~ SigaraDurum: r={r2:.3f}, p={p2:.3f}")
print(f"BisikletKullanimi ~ SigaraDurum: r={r3:.3f}, p={p3:.3f}")

# 3️⃣.3️⃣ Eğitim ve Test Seti Ayrımı
egitim = veriSeti.sample(frac=0.7, random_state=1)
test = veriSeti.drop(egitim.index)
print("Eğitim seti boyutu:", egitim.shape)
print("Test seti boyutu:", test.shape)

# 3️⃣.4️⃣ Çoklu Doğrusal Regresyon Modeli Kurulumu
model = smf.ols("kalpHastaligi ~ bisikletKullanimi + sigaraDurum", data=egitim).fit()

# Model Özeti
print(model.summary())

# Regresyon Denklemi
print("\n📌 Regresyon Denklemi:")
print(
    f"kalpHastaligi = {model.params['Intercept']:.3f} + "
    f"({model.params['bisikletKullanimi']:.3f}) * bisikletKullanimi + "
    f"({model.params['sigaraDurum']:.3f}) * sigaraDurum"
)

### 📊 Regresyon Modeli Çıktısının Yorumlanması

Model çıktısına göre, `bisikletKullanimi` ve `sigaraDurum` değişkenlerinin `kalpHastaligi` üzerindeki etkileri **istatistiksel olarak anlamlıdır**. Kurulan çoklu doğrusal regresyon modeli, yüksek düzeyde açıklayıcılığa sahiptir ve sağlık alanında kullanılabilecek güvenilir bir tahminleme imkânı sunmaktadır.

---

#### 🔹 Temel Regresyon Sonuçları

| Değişken           | Katsayı (β) | Std. Hata | t-değeri | p-değeri |  %95 Güven Aralığı   |
|--------------------|-------------|-----------|----------|----------|----------------------|
| Intercept          |  14.9346    |   0.094   | 158.300  |  0.000   |  14.749 - 15.120     |
| bisikletKullanimi  | -0.1997     |   0.002   | -119.859 |  0.000   |  -0.203 - -0.196     |
| sigaraDurum        |  0.1808     |   0.004   |  41.565  |  0.000   |   0.172 -  0.189     |

---

#### Yorumsal Analiz

- **Intercept (β₀ = 14.935):** Hem bisiklet kullanımı hem de sigara içme düzeyi sıfır olan bireylerde kalp hastalığı skoru yaklaşık 14.935’tir.
- **bisikletKullanimi (β₁ = -0.1997):** Bisiklet kullanımında her bir birimlik artış, kalp hastalığı değerini ortalama **0.1997 puan azaltmaktadır**.
- **sigaraDurum (β₂ = 0.1808):** Sigara içme düzeyinde her bir birimlik artış, kalp hastalığı değerini ortalama **0.1808 puan artırmaktadır**.
- **p-değerleri:** Tüm değişkenler için p < 0.05 olduğundan, **tüm katsayılar istatistiksel olarak anlamlıdır**.

---

####  Modelin Açıklayıcılığı

- **R² = 0.978** → Kalp hastalığı değişkenindeki varyansın %97.8’i model tarafından açıklanabilmektedir.
- **Adj. R² = 0.978** → Düzeltilmiş açıklayıcılık da çok yüksektir, bu da modelin overfitting yapmadığını gösterir.
- **F-istatistiği = 7670, p < 0.0001** → Modelin genel olarak anlamlı olduğunu göstermektedir.

---

#### Modelin Güvenilirliği ve Uygunluğu

- **Durbin-Watson = 2.091** → Artıklar arasında otokorelasyon olmadığını gösterir (ideal değer ≈ 2).
- **Jarque-Bera = 0.764, p = 0.682** → Artıklar normal dağılmıştır.
- **Skew = -0.114, Kurtosis = 3.028** → Veri simetrik ve normal dağılıma yakındır.

---

### ✅ Sonuç

Bu modele göre:

- 🚲 **Bisiklet kullanımı**, kalp hastalığını azaltan koruyucu bir faktördür.
- 🚬 **Sigara kullanımı**, kalp hastalığını artıran ciddi bir risk faktörüdür.
- 📊 Model hem **istatistiksel olarak anlamlı**, hem de **yüksek açıklayıcılığa** sahip güvenilir bir yapıya sahiptir.

Bu sayede bireylerin yaşam tarzı özelliklerine bakılarak **kalp hastalığı riski bilimsel doğrulukla tahmin edilebilir**.

---

### 🧮 Örnek Tahmin Hesaplama

Bir birey için:

- `bisikletKullanimi = 35`
- `sigaraDurum = 20`

ise, beklenen kalp hastalığı değeri aşağıdaki şekilde hesaplanır:

```
kalpHastaligi = 14.935 + (-0.1997 * 35) + (0.1808 * 20) ≈ 10.17
```

Bu sonuç, fiziksel aktivite düzeyinin artırılması ve sigara kullanımının azaltılmasının kalp sağlığı açısından ne denli önemli olduğunu sayısal olarak göstermektedir.





---


---



---



## 4️⃣ Modelin Performans Değerlendirmesi ve Hata Analizi

Bu adımda, oluşturulan doğrusal regresyon modelinin tahmin performansı, test veri seti üzerinden çeşitli hata metrikleriyle değerlendirilecektir. Amaç, modelin ne kadar başarılı bir şekilde tahmin yaptığını **nicel olarak ölçmektir**.

---

### 📐 Neden Hata Analizi Yapılır?

Her model belirli bir düzeyde hata içerir. Modelin ne kadar iyi genellenebileceği, bu hataların büyüklüğüne bağlıdır. Bu nedenle, aşağıdaki hata metrikleri üzerinden model değerlendirilir:

- **ME (Mean Error):** Ortalama tahmin hatası  
- **MAE (Mean Absolute Error):** Ortalama mutlak hata  
- **MPE (Mean Percentage Error):** Ortalama yüzde hata  
- **MAPE (Mean Absolute Percentage Error):** Ortalama mutlak yüzde hata  
- **MSE (Mean Squared Error):** Ortalama karesel hata  
- **RMSE (Root Mean Squared Error):** Kök ortalama karesel hata  

---

### 🔍 Bu Aşamada Yapılanlar

✅ Test veri seti üzerinde tahminler üretilmiştir  
✅ Gerçek ve tahmin edilen değerler karşılaştırılmıştır  
✅ Hatalar manuel olarak hesaplanmış ve sklearn ile karşılaştırılmıştır  
✅ Performans metrikleri yorumlanarak model başarısı analiz edilmiştir  




In [ ]:
# 4️⃣.1️⃣ Test setinden rastgele bir örnek için manuel tahmin
kalp_test1 = 14.935 + (-0.1997 * 40) + (0.1808 * 15)
print("🔹 Tek test örneği için tahmin edilen kalp hastalığı:", round(kalp_test1, 2))


# 4️⃣.2️⃣ Tüm test seti için model tahminleri
tahminler = model.predict(test[["bisikletKullanimi", "sigaraDurum"]])  # Tahmin edilen kalp hastalığı değerleri
print("\n🔹 İlk 5 Tahmin:\n", tahminler.head())
print("🔹 İlk 5 Gerçek Değer:\n", test.kalpHastaligi.head())


# 4️⃣.3️⃣ Gerçek ve tahminleri tek tabloya alıyoruz
performansVerisi = pd.DataFrame({
    "y_test": test.kalpHastaligi,
    "tahminler": tahminler
})


# 4️⃣.4️⃣ Hataları hesaplıyoruz
performansVerisi["error"] = performansVerisi.y_test - performansVerisi.tahminler
ME   = performansVerisi["error"].mean()
MAE  = np.abs(performansVerisi["error"]).mean()
MPE  = 100 * (performansVerisi["error"] / performansVerisi["y_test"]).mean()
MAPE = 100 * (np.abs(performansVerisi["error"] / performansVerisi["y_test"])).mean()
MSE  = (performansVerisi["error"]**2).mean()
RMSE = np.sqrt(MSE)


# 4️⃣.5️⃣ Manuel hesaplanan metriklerin yazdırılması
print("\n📊 Manuel Hesaplanan Hata Metrikleri:")
print(f"ME   : {ME:.4f}")
print(f"MAE  : {MAE:.4f}")
print(f"MPE  : {MPE:.2f}%")
print(f"MAPE : {MAPE:.2f}%")
print(f"MSE  : {MSE:.4f}")
print(f"RMSE : {RMSE:.4f}")


# 4️⃣.6️⃣ Sklearn ile aynı metriklerin hesaplanması
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae  = mean_absolute_error(test.kalpHastaligi, tahminler)
mse  = mean_squared_error(test.kalpHastaligi, tahminler)
rmse = np.sqrt(mse)  # Bu her versiyonla uyumludur

print("\n🔎 Sklearn ile Hesaplanan Hatalar:")
print(f"MAE  : {mae:.4f}")
print(f"MSE  : {mse:.4f}")
print(f"RMSE : {rmse:.4f}")


### 📊 Model Performans Sonuçlarının Yorumlanması

Regresyon modelinin test veri seti üzerindeki performansı hem **manuel hesaplamalarla** hem de `sklearn` kütüphanesi ile ölçülmüştür. Sonuçlar, modelin doğruluk seviyesini ve hata büyüklüğünü anlamak açısından önemlidir.

---

#### 📉 Manuel Hesaplanan Hata Metrikleri

| Gözlem | Tahmin Edilen Kalp Hastalığı | Gerçek Kalp Hastalığı |
|--------|-------------------------------|------------------------|
|   1    | 2.33                          | 2.85                   |
|   2    | 17.72                         | 17.18                  |
|   3    | 6.49                          | 6.82                   |
|   4    | 16.30                         | 15.85                  |
|   5    | 7.19                          | 6.43                   |

> Gözlemler arasındaki farklılıklar, tahmin hatalarının değerlendirilmesi için temel oluşturur.

---

#### 📉 Manuel Hesaplanan Hata Metrikleri

| Metrik | Değer     |
|--------|-----------|
| **ME** *(Ortalama Hata)*                 | -0.0146   |
| **MAE** *(Ortalama Mutlak Hata)*        | 0.5112    |
| **MPE** *(Ortalama Yüzde Hata)*         | -2.56%    |
| **MAPE** *(Ortalama Mutlak Yüzde Hata)* | 8.96%     |
| **MSE** *(Ortalama Karesel Hata)*       | 0.4212    |
| **RMSE** *(Kök Ortalama Karesel Hata)*  | 0.6490    |

---

#### 📊 Sklearn Kütüphanesi ile Hesaplanan Hatalar

| Metrik | Değer   |
|--------|---------|
| **MAE**  | 0.5112 |
| **MSE**  | 0.4212 |
| **RMSE** | 0.6490 |

> `sklearn` ile yapılan hesaplamalar, manuel metriklerle birebir örtüşmekte ve modelin hata analizini doğrulamaktadır.

---

###  Yorumsal Değerlendirme

- **ME** değeri oldukça düşük (-0.0146), bu durum modelin sistematik bir sapma içermediğini gösterir.  
- **MAE** ve **RMSE** değerleri modelin hatalarının genelde küçük olduğunu ve modelin istikrarlı tahminler yaptığını gösterir.  
- **MAPE = %8.96**, yani model ortalama %9 sapma ile tahmin yapmaktadır. Bu oran sağlık verisi için oldukça kabul edilebilir düzeydedir.  
- **MPE** negatif olduğundan, model genellikle tahminleri **bir miktar düşük** yapma eğilimindedir.

---

### ✅ Genel Sonuç

Model, bireylerin bisiklet kullanımı ve sigara içme durumuna göre kalp hastalığı düzeyini **güvenilir ve tutarlı** şekilde tahmin etmektedir. Elde edilen hata metrikleri modelin hem **doğruluğunu** hem de **genellenebilirliğini** desteklemektedir. Bu doğrultuda model, sağlık alanında karar destek sistemlerinde etkili biçimde kullanılabilir.


### 5️⃣ Regresyon Doğrusu: Gerçek ve Tahmin Edilen Değerlerin Görselleştirilmesi

---

Bu adımda, doğrusal regresyon modelinin **test veri seti üzerindeki başarı düzeyi**, grafiksel olarak analiz edilmektedir. Gerçek verilerle modelin tahmin ettiği değerler aynı eksende çizilerek görsel bir karşılaştırma sunulmuştur.

Modelin performansını yalnızca sayısal metriklerle değil, aynı zamanda **görsel olarak da değerlendirmek** oldukça önemlidir. Bu sayede:

- Gerçek değerler ile tahminler arasındaki farklar daha sezgisel bir biçimde fark edilir.  
- Modelin veriye ne kadar iyi uyum sağladığı daha net gözlemlenebilir.  
- Regresyon doğrusu, verinin genel trendini nasıl temsil ediyor, daha kolay yorumlanabilir.

---


In [ ]:
# 1. Sabit değişkenlerin ortalama değerlerini alalım
sabit_sigara = test["sigaraDurum"].mean()
sabit_bisiklet = test["bisikletKullanimi"].mean()

# 2. Bağımsız değişken eksen değerlerini oluşturalım
x_bisiklet = np.linspace(test["bisikletKullanimi"].min(), test["bisikletKullanimi"].max(), 100)
x_sigara = np.linspace(test["sigaraDurum"].min(), test["sigaraDurum"].max(), 100)

# 3. Tahmin değerlerini üretelim
y_bisiklet = model.predict(pd.DataFrame({
    "bisikletKullanimi": x_bisiklet,
    "sigaraDurum": sabit_sigara
}))

y_sigara = model.predict(pd.DataFrame({
    "bisikletKullanimi": sabit_bisiklet,
    "sigaraDurum": x_sigara
}))

# 4. Grafikler
plt.figure(figsize=(15, 6))

# 🚲 Bisiklet Kullanımı Grafiği
plt.subplot(1, 2, 1)
plt.scatter(test["bisikletKullanimi"], test["kalpHastaligi"], color="deepskyblue", alpha=0.5, label="Gerçek Değerler")
plt.plot(x_bisiklet, y_bisiklet, color="blue", linewidth=3, label="Regresyon Doğrusu")
plt.title("Kalp Hastalığına Etki: Bisiklet Kullanımı (Sigara Sabit)")
plt.xlabel("Bisiklet Kullanımı")
plt.ylabel("Kalp Hastalığı")
plt.legend()
plt.grid(True)

# 🚬 Sigara Kullanımı Grafiği
plt.subplot(1, 2, 2)
plt.scatter(test["sigaraDurum"], test["kalpHastaligi"], color="mediumseagreen", alpha=0.5, label="Gerçek Değerler")
plt.plot(x_sigara, y_sigara, color="darkgreen", linewidth=3, label="Regresyon Doğrusu")
plt.title("Kalp Hastalığına Etki: Sigara Kullanımı (Bisiklet Sabit)")
plt.xlabel("Sigara Kullanımı")
plt.ylabel("Kalp Hastalığı")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


## 📈 Grafik Analizi ve Yorumlama

Aşağıda yer alan iki ayrı grafik, **çoklu doğrusal regresyon modelinin** bağımsız değişkenler bazında nasıl çalıştığını görsel olarak sunmaktadır. Her grafik, ilgili değişkenin **kalp hastalığı üzerindeki etkisini** gösterirken diğer değişken **sabit tutulmuştur**.

---

### 1️⃣ Kalp Hastalığına Etki: Bisiklet Kullanımı (Sigara Sabit)

- 🔵 **Mavi Noktalar:** Gerçek test verisindeki kalp hastalığı gözlemlerini temsil eder.
- 🔷 **Mavi Regresyon Çizgisi:** Modelin, bisiklet kullanımı arttıkça kalp hastalığı değerlerinde nasıl bir değişim öngördüğünü gösterir. Sigara değişkeni sabit tutulmuştur.

#### 🔍 Yorum:
- Bisiklet kullanımı arttıkça kalp hastalığı riskinin azaldığı net bir şekilde görülmektedir.  
- Noktalar doğrusal çizgiye oldukça yakın dağılmıştır, bu da modelin bisiklet değişkenini iyi açıkladığını gösterir.  
- Bu durum, fiziksel aktivitenin kalp sağlığı üzerindeki **koruyucu etkisini** desteklemektedir.

---

### 2️⃣ Kalp Hastalığına Etki: Sigara Kullanımı (Bisiklet Sabit)

- 🟢 **Yeşil Noktalar:** Gerçek test verisindeki kalp hastalığı gözlemlerini temsil eder.
- 🟢 **Yeşil Regresyon Çizgisi:** Modelin, sigara kullanımı arttıkça kalp hastalığında nasıl bir değişim beklediğini gösterir. Bisiklet kullanımı sabit tutulmuştur.

#### 🔍 Yorum:
- Sigara kullanımı arttıkça kalp hastalığı değerlerinde **yukarı yönlü bir eğilim** mevcuttur.  
- Ancak veriler daha dağınık göründüğünden, bu değişkenin açıklayıcılığı bisiklet kullanımına göre daha zayıf kalmaktadır.  
- Yine de pozitif eğim, sigaranın kalp hastalığına olan **olumsuz etkisini** destekler niteliktedir.

---

### ✅ Genel Sonuç

- 🚲 Bisiklet kullanımı değişkeni, kalp hastalığını azaltmada **daha güçlü bir açıklayıcıdır**.  
- 🚬 Sigara kullanımı ise daha zayıf ama yine de istatistiksel olarak **anlamlı bir risk faktörü** olarak karşımıza çıkmaktadır.  
- 📊 Görseller, çoklu regresyon modelinin nasıl çalıştığını **değişken bazında ayrıştırarak sezgisel olarak anlatmak için güçlü bir araçtır**.
